In [ ]:
import jax
import genjax

console = genjax.pretty(show_locals=True)
key = jax.random.PRNGKey(314159)

In [ ]:
@genjax.gen
def g(x):
    m0 = genjax.trace("m0", genjax.bernoulli)(x)
    return m0


@genjax.gen
def h(x):
    m0 = genjax.trace("sub", g)(x)
    return m0

In [ ]:
key, tr = h.simulate(key, (0.3,))